In [ ]:
# Montar o google drive no coolab
from google.colab import drive
drive.mount('/content/drive')

## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from transformers import BertTokenizer, RobertaTokenizer, BertTokenizerFast, BertModel, BertForSequenceClassification
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset

from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix, f1_score
from sklearn import metrics

import gensim
from gensim.models import Word2Vec

import pickle
import re
import os
from tqdm import tqdm
import ast
import random


## Importando DataFrames

In [ ]:
df_train_use = pd.read_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/02. clean/train_clean_corpus.pkl')
df_test_use = pd.read_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/02. clean/test_clean_corpus.pkl') 

In [ ]:
df_train_use.head()

## Funções para métrica de avaliação f1-latency


Para cálculo da métrica de interesse f1-*latency* são necessárias algumas etapas representadas pelas funções abaixo.

In [ ]:
def value_p(k):
    """Get the penalty value for the F latency measure.

    Parameters
    ----------
    k : int
        Median number of posts from the positive users.

    Returns
    -------
    penalty : float
        Penalty to use.
    """
    return -(np.log(1 / 3) / (k - 1))


def f_penalty(k, _p):
    """Get the penalty of the current user delay.

    Parameters
    ----------
    k : int
        Current user delay.
    _p : float
        Penalty.

    Returns
    -------
    f_penalty : float
        Penalty latency.
    """
    return -1 + (2 / (1 + np.exp((-_p) * (k - 1))))


def speed(y_pred, y_true, d, p):
    """Get speed for every user correctly classified as positive."""
    penalty_list = [
        f_penalty(k=d[i], _p=p)
        for i in range(len(y_pred))
        if y_pred[i] == 1 and y_true[i] == 1
    ]

    if len(penalty_list) != 0:
        return 1 - np.median(penalty_list)
    else:
        return 0.0


def f_latency(labels, true_labels, delays, penalty):
    """F latency metric.

    Metric proposed by Sadeque and others in [1]_.

    Parameters
    ----------
    labels : list of int
        Predicted label for each user.
    true_labels : list of int
        True label for each user.
    delays : list of int
        Decision delay for each user.
    penalty : float
        Penalty. Defines how quickly the penalty should increase.

    Returns
    -------
    f_latency_metric : float
        F latency measure.

    References
    ----------
    .. [1] `Sadeque, F., Xu, D., & Bethard, S. (2018, February). Measuring the
        latency of depression detection in social media. In Proceedings of the
        Eleventh ACM International Conference on Web Search and Data Mining
        (pp. 495-503).`_
    """
    f1_score = metrics.f1_score(y_pred=labels, y_true=true_labels, average="binary")
    speed_value = speed(y_pred=labels, y_true=true_labels, d=delays, p=penalty)

    return f1_score * speed_value

## Treinando Word2Vec com Regressão Logística

### Treinando com o melhor classificador

In [ ]:
# Transforme os textos em uma lista de listas de palavras
sentences = df_train_use['Text'].apply(lambda x: x.split()).tolist()

# Treine o modelo Word2Vec
model_w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Função para calcular a média dos vetores das palavras
def text_vector(text, model):
    words = text.split()
    word_vecs = [model.wv[word] for word in words if word in model.wv]
    if len(word_vecs) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vecs, axis=0)

X_train_tf = df_train_use['Text'].apply(lambda x: text_vector(x, model_w2v)).tolist()
X_train_tf = np.array(X_train_tf)
y_train_tf = df_train_use['Diagnosed_YN']

In [ ]:
# Treinando a Regressão Linear
clf = LogisticRegression(random_state=42)
clf.fit(X_train_tf, y_train_tf)

In [ ]:
y_pred = clf.predict(X_train_tf)

In [ ]:
print(classification_report(y_train_tf, y_pred))

In [ ]:
save_path = '/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/02. model/01. W2V.reglog/'
# Salvar o modelo no arquivo .pkl
with open(os.path.join(save_path, 'modelo.pkl'), 'wb') as f:
    pickle.dump(clf, f)

## Política DMC - Momento de Decisão da Classificação
Os parâmetros usados nesta política foram de limiar = 0.5 e número mínimo de mensagens lidas de 10.

#### Utilizando a política como está descrita no artigo

A política foi inspirada no trabalho descrito em Loyola et al. (2022) com as camadas de Decisão do Momento de Classificação (DMC) e Classificação Parcial da Informação (CPI) com adaptações necessárias para o português no córpus SetembroBR.

In [ ]:
## Carregando o modelo de Regressão Logística a partir do arquivo .pkl
# with open('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/02. model/01. W2V.reglog/modelo.pkl', 'rb') as f:
#     clf = pickle.load(f)

In [ ]:
# Carregar o bag-of-words do arquivo .pkl
# with open('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/02. model/01. W2V.reglog/bow.pkl', 'rb') as f:
#     count_vect, tf_transformer = pickle.load(f)

Preparação dos dados de teste

In [ ]:
menor_timeline = df_test_use['Qt_Messages'].min()

def min_timeline(lista):
    return lista[- menor_timeline:]

df_test_use['List_CleanText_min'] = df_test_use['List_CleanText'].apply(min_timeline)
df_test_use.head()

In [ ]:
# Parâmetros
delta = 0.5
n = 10

# Criando objetos auxiliares
qtd_msgs_lidas = list(df_test_use['Qt_Messages'])
df_test_use['prediction'] = 0

# Avaliando a previsão e calculando a quantidade de mensagens lidas
for i, lista in tqdm(enumerate(df_test_use['List_CleanText_min'])):
    # Concatenando as mensagens em um único documento
    concatenated_messages = ['$END_OF_POST$'.join(lista[j:j+n]) for j in range(0, len(lista))]

    # Usando o Word2Vec
    # Criar vetores para cada mensagem concatenada
    all_vectors = []
    vectors = [text_vector(message, model_w2v) for message in concatenated_messages]
    all_vectors.append(vectors)

    X_train_tf_teste = [np.mean(vectors, axis=0) for vectors in all_vectors]
    X_train_tf_teste = np.array(X_train_tf_teste)

    # Prevendo usando o modelo SVM
    y_pred = clf.predict(X_train_tf_teste)
    prob = clf.predict_proba(X_train_tf_teste)

    # Encontrando o índice da primeira mensagem que satisfaz os critérios
    idx = np.argmax((y_pred == 1) & (prob.max(axis=1) >= delta))

    # Atualizando o dataframe e a quantidade de mensagens lidas
    if (y_pred[idx] == 1) and (prob[idx].max() >= delta):
        df_test_use.loc[i, 'prediction'] = 1
        qtd_msgs_lidas[i] = (idx + n)

df_test_use['qtd_msgs_lidas'] = qtd_msgs_lidas


In [ ]:
df_test_use.to_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/03. prediction/prediction_w2vreglog_MinTL.pkl')

### Avaliando sistema

In [ ]:
y_train_teste_1 = df_test_use['Diagnosed_YN']
y_pred_teste_1 = df_test_use['prediction']
print(classification_report(y_train_teste_1, y_pred_teste_1))

In [ ]:
f1_latency = f_latency(df_test_use['prediction'], df_test_use['Diagnosed_YN'], df_test_use['qtd_msgs_lidas'], 0.0078)
f1_latency

In [ ]:
# Confusion matrix
cm = confusion_matrix(df_test_use['Diagnosed_YN'], df_test_use['prediction'])

# Exibindo a matriz de confusão
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

Tempo de antecipação do risco

In [ ]:
df_test_use.head()

In [ ]:
from datetime import datetime

# Função para converter a lista de datas em objetos datetime
def converter_datas(lista_datas_str):
    # Remover os colchetes e espaços em branco
    # lista_datas_str = lista_datas_str.strip('[]').replace("'", "").split(', ')
    # Converter cada data para datetime
    return [pd.to_datetime(data_str, format='%d-%b-%Y %H:%M') for data_str in lista_datas_str]


# Função para calcular a diferença em dias entre duas datas
def calcular_diferenca_dias(data1, data2):
    delta = data1 - data2
    return delta.days

# Aplicando a conversão de datas ao DataFrame
df_test_use['List_Timeline'] = df_test_use['List_Timeline'].apply(converter_datas)

# Aplicando a função ao DataFrame
df_test_use['ultima_data_tweet_lido'] = df_test_use.apply(lambda row: row['List_Timeline'][row['qtd_msgs_lidas'] - 2], axis=1)
df_test_use['ultima_data_tweet_lista'] = df_test_use.apply(lambda row: row['List_Timeline'][-1], axis=1)
df_test_use['dias_antecipados'] = df_test_use.apply(lambda row: calcular_diferenca_dias(row['ultima_data_tweet_lista'], row['ultima_data_tweet_lido']), axis=1)

# df_test_use.head()

In [ ]:
dias_antecipacao_geral = df_test_use['dias_antecipados'].mean()
dias_antecipacao_pos = df_test_use[df_test_use['Diagnosed_YN']==1]['dias_antecipados'].mean()
print(f'Média de dias de antecipação de risco geral: {dias_antecipacao_geral}')
print(f'Média de dias de antecipação de risco da classe positiva: {dias_antecipacao_pos}')

In [ ]:
df_test_use.to_pickle('/content/drive/MyDrive/Mestrado/Dissertação/Experimentos/anxiety/01. data/03. prediction/prediction_w2vreglog_MinTL.pkl')